In [91]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [92]:
data = load_breast_cancer()

In [93]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [94]:
df.shape

(569, 31)

In [95]:
X = df.drop(['target'], axis=1)
y = df['target'].astype(float)

In [96]:
df.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target'],
      dtype='object')

In [97]:
model = DecisionTreeClassifier()

In [98]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=2023)
scores = cross_val_score(model, X, y, scoring = "recall", cv=cv, n_jobs=-1)
print(f"Mean recall: {np.mean(scores)} {np.std(scores)}")
print(scores)

Mean recall: 0.9400625978090765 0.02587151952992362
[0.94366197 0.87323944 0.94444444 0.95833333 0.94366197 0.91549296
 0.90140845 0.90277778 0.93055556 0.92957746 0.97183099 0.94366197
 0.95833333 0.93055556 0.91549296 0.97183099 0.94366197 0.97222222
 0.91666667 0.95774648 0.94366197 0.98591549 0.93055556 0.94444444
 0.97183099]


In [99]:
len(scores), max(scores), min(scores)

(25, 0.9859154929577465, 0.8732394366197183)

## Version with subset of features

In [100]:
from itertools import product
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [101]:
n_cols = X.shape[1]
best_subset, best_score = None, 0.0
n_cols

30

In [102]:
import random

# create an empty list to store the result
feature_list_random = []
n = 0
# generate 100 lists
for i in range(100):
    # create an empty list to store the boolean values
    sub_list = []

    # generate 30 boolean values
    for j in range(30):
        sub_list.append(random.choice([True, False]))
    n=n+1
    # add the sub_list to the result list
    print(n)
    feature_list_random.append(sub_list)

# print the result
# print(result)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [103]:
len(feature_list_random[1])

30

In [104]:
# for subset in product([True, False], repeat = n_cols):
for subset in feature_list_random:
    ix = [i for i, x in enumerate(subset) if x]

    if len(ix) == 0:
        continue

    X_new = X.iloc[:, ix]

    model = DecisionTreeClassifier()
    cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=2023)
    scores = cross_val_score(model, X, y, scoring = "recall", cv=cv, n_jobs=-1)

    result = mean(scores)
    print(f"Subset: {ix} Result: {result}")

    if best_score is None or result >= best_score:
        best_subset, best_score = ix, result
    # break

print('Achou!')
print(f"Best Subset: {best_subset} Best Score: {best_score}")

Subset: [0, 1, 2, 3, 8, 9, 12, 13, 17, 21, 22, 25, 26, 27] Result: 0.935550185173561
Subset: [1, 4, 5, 10, 11, 12, 14, 15, 16, 20, 21, 22, 23, 24, 25, 28] Result: 0.929932207645471
Subset: [0, 3, 5, 6, 8, 10, 13, 15, 22, 23, 24, 26, 29] Result: 0.9383591739376058
Subset: [1, 4, 6, 8, 9, 11, 13, 14, 17, 18, 21, 22, 26, 27, 28, 29] Result: 0.935550185173561
Subset: [2, 5, 7, 9, 10, 11, 14, 15, 17, 22, 23, 25, 27, 28] Result: 0.9439771514656958
Subset: [0, 2, 4, 7, 16, 17, 18, 19, 20, 22, 24, 25, 26, 27, 28] Result: 0.932741196409516
Subset: [1, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 20, 21, 22, 23, 25, 29] Result: 0.935550185173561
Subset: [1, 2, 6, 7, 8, 9, 11, 13, 15, 16, 19, 21, 22, 24, 25, 28] Result: 0.9411681627016508
Subset: [0, 1, 3, 6, 7, 8, 9, 11, 12, 16, 17, 18, 19, 20, 23, 25, 28] Result: 0.935550185173561
Subset: [1, 5, 7, 9, 11, 12, 13, 14, 16, 17, 19, 23, 24, 26, 27, 29] Result: 0.9271232188814261
Subset: [4, 5, 6, 8, 9, 10, 14, 16, 18, 20, 22, 24, 25, 26, 27] Result: 0.9

In [105]:
def column_names_by_indexes(column_names, indexes):
    return [column_names[i] for i in indexes]

In [106]:
column_names_by_indexes(df.columns, best_subset)

['mean perimeter',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean fractal dimension',
 'perimeter error',
 'area error',
 'concave points error',
 'symmetry error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst compactness',
 'worst concave points',
 'worst symmetry']

In [107]:
df.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target'],
      dtype='object')

In [108]:
2 ** 30

1073741824

## Using Hill Climbing as the optimizer

In [114]:
# objective function
def objective(X, y, subset):
	# convert into column indexes
	ix = [i for i, x in enumerate(subset) if x]
	# check for now column (all False)
	if len(ix) == 0:
		return 0.0
	# select columns
	X_new = X.iloc[:, ix]
	# define model
	model = DecisionTreeClassifier()
	# evaluate model
	scores = cross_val_score(model, X_new, y, scoring='accuracy', cv=3, n_jobs=-1)
	# summarize scores
	result = mean(scores)
	return result, ix



In [115]:
# mutation operator
def mutate(solution, p_mutate):
	# make a copy
	child = solution.copy()
	for i in range(len(child)):
		# check for a mutation
		if np.random.rand() < p_mutate:
			# flip the inclusion
			child[i] = not child[i]
	return child

In [116]:
# hill climbing local search algorithm
def hillclimbing(X, y, objective, n_iter, p_mutate):
	# generate an initial point
	solution = np.random.choice([True, False], size=X.shape[1])
	# evaluate the initial point
	solution_eval, ix = objective(X, y, solution)
	# run the hill climb
	for i in range(n_iter):
		# take a step
		candidate = mutate(solution, p_mutate)
		# evaluate candidate point
		candidate_eval, ix = objective(X, y, candidate)
		# check if we should keep the new point
		if candidate_eval >= solution_eval:
			# store the new point
			solution, solution_eval = candidate, candidate_eval
		# report progress
		# print('>%d f(%s) = %f' % (i+1, len(ix), solution_eval))
		print(f"i:{i+1}, len: {len(ix)}, solution eval: {solution_eval}")
	return solution, solution_eval


In [121]:
n_iter = 300
p_mut = 2.0/100.0

In [122]:
# perform the hill climbing search
subset, score = hillclimbing(X, y, objective, n_iter, p_mut)
# convert into column indexes
ix = [i for i, x in enumerate(subset) if x]
print(f"ix: {ix}, len(ix): {len(ix)}, score: {score}")

i:1, len: 16, solution eval: 0.9296946068875894
i:2, len: 16, solution eval: 0.9296946068875894
i:3, len: 16, solution eval: 0.9296946068875894
i:4, len: 16, solution eval: 0.9296946068875894
i:5, len: 16, solution eval: 0.9297224542838577
i:6, len: 17, solution eval: 0.9297224542838577
i:7, len: 15, solution eval: 0.9297224542838577
i:8, len: 15, solution eval: 0.9297224542838577
i:9, len: 15, solution eval: 0.9297224542838577
i:10, len: 17, solution eval: 0.9349856121785947
i:11, len: 17, solution eval: 0.9349856121785947
i:12, len: 18, solution eval: 0.9349856121785947
i:13, len: 15, solution eval: 0.9349856121785947
i:14, len: 16, solution eval: 0.9349856121785947
i:15, len: 17, solution eval: 0.9349856121785947
i:16, len: 18, solution eval: 0.9349856121785947
i:17, len: 18, solution eval: 0.9349856121785947
i:18, len: 18, solution eval: 0.9349856121785947
i:19, len: 17, solution eval: 0.9349856121785947
i:20, len: 17, solution eval: 0.9349856121785947
i:21, len: 17, solution eval:

In [123]:
print(f"ix: {ix}, len(ix): {len(ix)}, score: {score}")

ix: [2, 6, 9, 12, 13, 15, 16, 17, 19, 20, 21, 23, 24, 26, 28], len(ix): 15, score: 0.9560660911538105


In [124]:
column_names_by_indexes(df.columns, ix)

['mean perimeter',
 'mean concavity',
 'mean fractal dimension',
 'perimeter error',
 'area error',
 'compactness error',
 'concavity error',
 'concave points error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst area',
 'worst smoothness',
 'worst concavity',
 'worst symmetry']